# Py City School Analysis

This is an analysis to help determine school performance based on student's math and reading scores across schools within the district. It will analyze test scores, school size, and budget.  It will include both district and charter schools.  The goal is to summarize obvious trends that will help in assessing future school budgets and priorities. 

### Trends

1. There are 15 schools that were included in this study which comprised of 8 charter schools and 7 district schools.
2. The current average budget per school is $1,643,295.30.
3. There are significantly more students in district schools versus charter schools.
4. It costs more per student to educate a child in a district school.
5. On the average, students in charter schools score higher in Math and Science than those in the district schools.
6. Student's overall performance are significantly better in charter schools (95%) than those in district schools (76%) 
7. The top 5 performing schools are all charter schools and the bottom 5 performing schools are all district schools

### Summary

The study shows that on average, students in the charter schools perform significantly better that those that attended the district schools. Trends show that the cost per student does not signify better performance.  It does show, though, that the student's performance fair much better in smaller school sizes.



In [204]:
import pandas as pd
import numpy as np
import locale

locale.setlocale(locale.LC_ALL, 'en-US')

# declare path for student file
student_file = "Resources/students_complete.csv"

# declare path for school file
school_file = "Resources/schools_complete.csv"

#open files
student_pd = pd.read_csv(student_file)
school_pd = pd.read_csv(school_file)

school_pd.columns

# Rename columns for readability
renamed_school_pd = school_pd.rename(columns={
    "name": "School Name",
    "type": "School Type"
})


In [205]:
#renamed columns for merging
renamed_student_pd = student_pd.rename(columns={
    "name": "Student Name",
    "school" : "School Name"
})

# District Summary

In [206]:
# get total schools
totalSchools = renamed_school_pd['School Name'].count()
df = pd.DataFrame([totalSchools], columns=['Total Schools'])

In [207]:
# get total students
totalStudents = renamed_student_pd['School Name'].count()
df["Total Students"] = locale.format("%d", totalStudents, grouping=True)
# df

In [208]:
# total school budget
totalBudget = np.round(pd.to_numeric(renamed_school_pd['budget']).sum(), decimals=2)
# totalBudget
df["Total Budget"] = locale.currency(totalBudget, grouping=True)

In [209]:
# average math score
averageMathScore = renamed_student_pd["math_score"].mean()
df["Average Math Score"] = averageMathScore

In [210]:
# average reading score
averageReadingScore = renamed_student_pd["reading_score"].mean()
df["Average Reading Score"] = averageReadingScore
# df

In [211]:
# get the number of students that scored 70 or more in Math
totalPassingMath = renamed_student_pd.loc[(renamed_student_pd["math_score"] >= 70)]["Student Name"].count()
percentPassingMath = (totalPassingMath / totalStudents) * 100
df["% Passing Math"] = percentPassingMath

In [212]:
# get the number of students that scored 70 or more in Reading
# totalPassingReading = renamed_student_pd.loc[(renamed_student_pd["reading_score"] >= 70)]["Student Name"].count()
totalPassingReading = renamed_student_pd[(renamed_student_pd["reading_score"] >= 70)]["Student Name"].count()
percentPassingReading = (totalPassingReading / totalStudents) * 100

df["% Passing Reading"] = totalPassingReading

In [213]:
# compute the average of the number of students that scored 70 or more in Math and those that scored 70 or more in Reading 
overallPassingRate = ((totalPassingMath + totalPassingReading) / 2) / totalStudents * 100

df["% Overall Passing Rate"] = overallPassingRate
df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,33610,80.393158


# School Summary

In [214]:
merged_pd = renamed_school_pd.merge(renamed_student_pd, on='School Name', how='outer')


In [215]:
# find total students, total budget, average math score, average reading score
# df = merged_pd.groupby(['School Name', 'School Type']).aggregate({'Student ID':'count','budget':'max', 'math_score': 'mean', 'reading_score': 'mean'})
df = merged_pd.groupby(['School Name']).aggregate({'School Type':'min','Student ID':'count','budget':'max', 'math_score': 'mean', 'reading_score': 'mean'})


In [216]:
# Find the total # of students by School/District with math scores > 70 
TotalPassingMath = merged_pd[merged_pd['math_score'] >= 70].groupby(['School Name'])['math_score'].count()
df['Total Passing Math'] = TotalPassingMath


In [217]:
# Find the total # of students by School/District with reading score > 70
TotalPassingReading = merged_pd[merged_pd['reading_score'] >= 70].groupby(['School Name'])['reading_score'].count()
df['Total Passing Reading'] = TotalPassingReading

In [218]:
# calculate budget per student
df["Per Student Budget"] = df['budget']/df['Student ID']
# df.head()

In [219]:
df["% Passing Math"] = df["Total Passing Math"]/df["Student ID"] * 100

In [220]:
df["% Passing Reading"] = df["Total Passing Reading"]/df["Student ID"] * 100

In [221]:
df["% Overall Passing Rate"] = ((df["Total Passing Math"] + df["Total Passing Reading"])/2)/df['Student ID'] * 100

In [222]:
# rename columns
df_renamed = df.rename(columns = {
    "budget":"Total School Budget",
    "Student ID": "Total Students",
    "math_score": "Average Math Score",
    "reading_score" : "Average Reading Score"
    
})

In [223]:
# select columns to show in report 
df_final = df_renamed[["School Type","Total Students","Total School Budget","Per Student Budget","Average Math Score","Average Reading Score","% Passing Math", "% Passing Reading","% Overall Passing Rate"]]

In [224]:
# Final Report
df_final.style.format({"Total School Budget": "${:,.2f}", "Per Student Budget": "${:,.2f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.0484,81.034,66.6801,81.9333,74.3067
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.0619,83.9758,94.1335,97.0398,95.5867
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7118,81.158,65.9885,80.7392,73.3639
Ford High School,District,2739,"$1,763,916.00",$644.00,77.1026,80.7463,68.3096,79.299,73.8043
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.3515,83.8168,93.3924,97.139,95.2657
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.2898,80.9344,66.753,80.863,73.808
Holden High School,Charter,427,"$248,087.00",$581.00,83.8033,83.815,92.5059,96.2529,94.3794
Huang High School,District,2917,"$1,910,635.00",$655.00,76.6294,81.1827,65.6839,81.3164,73.5002
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.0725,80.9664,66.0576,81.2224,73.64


# Top Performing School ( By Passing Rate)

In [225]:
# Top 5 performing schools final report
df_sorted_top_final = df_final.sort_values('% Overall Passing Rate', ascending=False).head()
# Final Report
df_sorted_top_final.style.format({"Total School Budget": "${:,.2f}", "Per Student Budget": "${:,.2f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.0619,83.9758,94.1335,97.0398,95.5867
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.4183,83.8489,93.2722,97.3089,95.2905
Pena High School,Charter,962,"$585,858.00",$609.00,83.8399,84.0447,94.5946,95.9459,95.2703
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.3515,83.8168,93.3924,97.139,95.2657
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.2742,83.9895,93.8677,96.5396,95.2037


## Bottom Performing Schools (By Passing Rate)

In [226]:
# Bottom 5 performing schools final report
df_sorted_bottom_final = df_final.sort_values('% Overall Passing Rate', ascending=True).head()
df_sorted_bottom_final.style.format({"Total School Budget": "${:,.2f}", "Per Student Budget": "${:,.2f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.8427,80.7447,66.3666,80.2201,73.2933
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7118,81.158,65.9885,80.7392,73.3639
Huang High School,District,2917,"$1,910,635.00",$655.00,76.6294,81.1827,65.6839,81.3164,73.5002
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.0725,80.9664,66.0576,81.2224,73.64
Ford High School,District,2739,"$1,763,916.00",$644.00,77.1026,80.7463,68.3096,79.299,73.8043


## Math Scores By Grade

In [227]:
# created a numeric field called level to sort the grade column correctly
student_sorted_pd = pd.DataFrame(student_pd)
level = pd.to_numeric(student_sorted_pd["grade"].str.split("th").str[0])
student_sorted_pd["level"] = level
student_sorted_pd.sort_values(["school","level","grade"], ascending=True, inplace=True)

In [228]:
# calculated the average math score and inverse the columns
student_math_df = student_sorted_pd.groupby(['school','level']).aggregate({'math_score': 'mean'}).unstack(level=1)

In [229]:
# rename multi level column names Level 0 (math_score)
student_math_df.columns.set_levels(['Average Math Score'], level=0, inplace=True)

# rename multi level column names Level 1 (level) to an alphanumeric column name
student_math_df.columns.set_levels(['9th','10th','11th','12th'], level=1, inplace=True)

# rename rows index name
student_math_df.index.names = ['School Name']

# rename columns index name
student_math_df.columns.names = ['','']
student_math_df

Average Math Score                                 
                                     9th       10th       11th       12th
School Name                                                              
Bailey High School             77.083676  76.996772  77.515588  76.492218
Cabrera High School            83.094697  83.154506  82.765560  83.277487
Figueroa High School           76.403037  76.539974  76.884344  77.151369
Ford High School               77.361345  77.672316  76.918058  76.179963
Griffin High School            82.044010  84.229064  83.842105  83.356164
Hernandez High School          77.438495  77.337408  77.136029  77.186567
Holden High School             83.787402  83.429825  85.000000  82.855422
Huang High School              77.027251  75.908735  76.446602  77.225641
Johnson High School            77.187857  76.691117  77.491653  76.863248
Pena High School               83.625455  83.372000  84.328125  84.121547
Rodriguez High School          76.859966  76.612500  76.395626  77.690748
Shelton High School            83.420755  82.917411  83.383495  83.778976
Thomas High School             83.590022  83.087886  83.498795  83.497041
Wilson High School             83.085578  83.724422  83.195326  83.035794
Wright High School             83.264706  84.010288  83.836782  83.644986

## Reading Scores by Grade

In [230]:
student_reading_df = student_sorted_pd.groupby(['school','level']).aggregate({'reading_score': 'mean'}).unstack(level=1)

##### rename multi level column names Level 0 (math_score)
student_reading_df.columns.set_levels(['Average Reading Score'], level=0, inplace=True)

# rename multi level column names Level 1 (level) to an alphanumeric column name
student_reading_df.columns.set_levels(['9th','10th','11th','12th'], level=1, inplace=True)

# rename rows index name
student_reading_df.index.names = ['School Name']

# rename columns index name
student_reading_df.columns.names = ['','']
student_reading_df

## Scores by School Spending

In [231]:
# # use describe to find out 25%, 50%, 75%, max bins
firstvalue = df_final['Per Student Budget'].describe().loc["25%"]
secondvalue = df_final['Per Student Budget'].describe().loc["50%"]
thirdvalue = df_final['Per Student Budget'].describe().loc["75%"]
fourthvalue = df_final['Per Student Budget'].describe().loc["max"]

# create row names
firstbucket = " < " + locale.format("%d", (firstvalue), grouping=False)
secondbucket = locale.format("%d", (firstvalue), grouping=False) + " - " + locale.format("%d", (secondvalue), grouping=False)                             
thirdbucket = locale.format("%d", (secondvalue+1), grouping=False) + " - " + locale.format("%d", (thirdvalue), grouping=False)
fourthbucket = locale.format("%d", (thirdvalue+1), grouping=False) + " - " + locale.format("%d", (fourthvalue), grouping=False)


In [232]:
# copy data frame
scoresBySpending_df = df_final.copy()

In [233]:
# assign a bucket number to each of the school
scoresBySpending_df.loc[(scoresBySpending_df['Per Student Budget'] < firstvalue), 'Spending Ranges (per Student)'] = firstbucket
scoresBySpending_df.loc[(scoresBySpending_df['Per Student Budget'] >= firstvalue) & (scoresBySpending_df['Per Student Budget'] <= secondvalue), 'Spending Ranges (per Student)'] = secondbucket
scoresBySpending_df.loc[(scoresBySpending_df['Per Student Budget'] >= secondvalue+1) & (scoresBySpending_df['Per Student Budget'] <= thirdvalue), 'Spending Ranges (per Student)'] = thirdbucket
scoresBySpending_df.loc[(scoresBySpending_df['Per Student Budget'] > thirdvalue), 'Spending Ranges (per Student)'] = fourthbucket

In [234]:
# Group schools by budget per student 
scoresBySpending_df.groupby('Spending Ranges (per Student)').aggregate({"Average Math Score":"mean", "Average Reading Score":"mean", "% Passing Math":"mean","% Passing Reading":"mean","% Overall Passing Rate":"mean"})


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (per Student),,,,,
< 591,83.455399,83.933814,93.460096,96.610877,95.035486
591 - 628,81.899826,83.155286,87.133538,92.718205,89.925871
629 - 641,78.990942,81.917212,75.209078,86.089386,80.649232
642 - 655,77.023555,80.957446,66.701010,80.675217,73.688113


## Scores by School Size

In [235]:
# use describe to find out first(25%), seond(the difference of 50% and 75% plus 50%), third(max) bins
firstvalue = df_final['Total Students'].describe().loc["25%"]
secondvalue = df_final['Total Students'].describe().loc["50%"] + ((df_final['Total Students'].describe().loc["50%"] - df_final['Per Student Budget'].describe().loc["75%"]) / 2)
thirdvalue = df_final['Total Students'].describe().loc["max"]

# create row names
firstbucket = "Small (<" + locale.format("%d", (firstvalue), grouping=False) + ")"
secondbucket = "Medium (" + locale.format("%d", (firstvalue), grouping=False) + "-" + locale.format("%d", (secondvalue), grouping=False) + ")"                            
thirdbucket = "Large (" + locale.format("%d", (secondvalue+1), grouping=False) + "-" + locale.format("%d", (thirdvalue), grouping=False) + ")"


In [236]:
# copy data frame
scoresBySize_df = df_final.copy()

In [237]:
# assign a bucket number to each of the school
scoresBySize_df.loc[(scoresBySize_df['Total Students'] < firstvalue), 'School Bucket'] = 1
scoresBySize_df.loc[(scoresBySize_df['Total Students'] >= firstvalue) & (scoresBySpending_df['Total Students'] <= secondvalue), 'School Bucket'] = 2
scoresBySize_df.loc[(scoresBySize_df['Total Students'] >= secondvalue+1) & (scoresBySpending_df['Total Students'] <= thirdvalue), 'School Bucket'] = 3

# list the string equivalent of the bucket
scoresBySize_df.loc[(scoresBySize_df['School Bucket'] == 1), 'School Size'] = firstbucket
scoresBySize_df.loc[(scoresBySize_df['School Bucket'] == 2), 'School Size'] = secondbucket
scoresBySize_df.loc[(scoresBySize_df['School Bucket'] == 3), 'School Size'] = thirdbucket


In [238]:
# create a data frame with the row index order
index = [firstbucket,secondbucket,thirdbucket]
BySize_created_df = pd.DataFrame(index=index)

# Average Math column
selected = scoresBySize_df.groupby('School Size')['Average Math Score'].mean()
BySize_created_df ['Average Math Score'] = selected
# Average Reading column
selected = scoresBySize_df.groupby('School Size')['Average Reading Score'].mean()
BySize_created_df ['Average Reading Score'] = selected
# % Passing Math
selected = scoresBySize_df.groupby('School Size')['% Passing Math'].mean()
BySize_created_df ['% Passing Math'] = selected
# % Passing Reading
selected = scoresBySize_df.groupby('School Size')['% Passing Reading'].mean()
BySize_created_df ['% Passing Reading'] = selected
# % Overall Passing Rate
selected = scoresBySize_df.groupby('School Size')['% Overall Passing Rate'].mean()
BySize_created_df ['% Overall Passing Rate'] = selected

BySize_created_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Small (<1698),83.603261,83.881343,93.441248,96.661677,95.051462
Medium (1698-3103),80.545935,82.676142,82.169092,89.628554,85.898823
Large (3104-4976),77.063340,80.919864,66.464293,81.059691,73.761992


## Scores by School Type

In [239]:
# copy data frame
scoresByType_df = df_final.copy()

In [240]:
# Group schools by student population
scoresByType_df.groupby('School Type').aggregate({"Average Math Score":"mean", "Average Reading Score":"mean", "% Passing Math":"mean","% Passing Reading":"mean","% Overall Passing Rate":"mean"})


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
